<a href="https://colab.research.google.com/github/Milham10/PracticeML/blob/main/SentimentTweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset = '/content/drive/MyDrive/dataset/training.1600000.processed.noemoticon.csv.zip'

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [5]:
names_coloumn = ['target', 'ids', 'date', 'flag', 'user', 'text']

In [6]:
tweet = pd.read_csv(dataset, names=names_coloumn , encoding='latin-1')

In [7]:
tweet.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


*   0 = Negative
*   2 = Netral
*   4 = Positive

Harus di decode karena cmn angka labelnya







In [10]:
decode_map = {0: "NEGATIVE", 2: "NEUTRAL", 4: "POSITIVE"}
def decode_sentiment(label):
    return decode_map[int(label)]

In [11]:
tweet.target = tweet.target.apply(lambda x: decode_sentiment(x))

# Bersihkan text

In [12]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
import re

In [19]:
stop_word = stopwords.words("english")

In [33]:
sentences = tweet['text']

In [34]:
sentences.head()

0    @switchfoot http://twitpic.com/2y1zl - Awww, t...
1    is upset that he can't update his Facebook by ...
2    @Kenichan I dived many times for the ball. Man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object

In [37]:
def clean_tweet(text):
    text = re.sub(r"#\w+", " ", text)            # remove hashtags
    text = re.sub(r"@\w+", " ",text)             # remove mentions
    text = re.sub(r"http\S+", " ", text)         # remove urls
    text = re.sub(r"[^a-zA-Z]", " ", text)       # remove non-words (digits, punctuations etc)
    text = text.lower().strip()                  # convert tweet to lowercase and strip
    
    text = " ".join([word for word in text.split() if not word in stop_word])           # remove stop words    
    
    text = " ".join(nltk.word_tokenize(text))           # tokenize text
      
    return text

In [36]:
clean_tweet(sentences)

TypeError: ignored

In [38]:
tweet.text = tweet.text.apply(lambda x: clean_tweet(x))

In [39]:
tweet['text'].head()

0         awww bummer shoulda got david carr third day
1    upset update facebook texting might cry result...
2    dived many times ball managed save rest go bounds
3                     whole body feels itchy like fire
4                                     behaving mad see
Name: text, dtype: object

# Split Dataset

In [45]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [41]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_portion = .8

In [43]:
sentences = tweet['text']
labels = tweet['target']

In [44]:
training_sentences, testing_sentences, training_labels, testing_labels = train_test_split(sentences, labels, train_size=training_portion, shuffle=False)

In [46]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

In [47]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [48]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
train_label_seq = label_tokenizer.texts_to_sequences(training_labels)
train_label_seq_np = np.array(train_label_seq) - 1

In [49]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
test_label_seq = label_tokenizer.texts_to_sequences(testing_labels)
test_label_seq_np = np.array(test_label_seq) - 1

In [55]:
model = tf.keras.Sequential([
        # YOUR CODE HERE.
        # YOUR CODE HERE. DO not change the last layer or test may fail
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(6, activation='softmax')
    ])

In [56]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           16000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 24)                408       
                                                                 
 dense_3 (Dense)             (None, 6)                 150       
                                                                 
Total params: 16,558
Trainable params: 16,558
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(training_padded, train_label_seq_np, epochs=10, validation_data=(testing_padded, test_label_seq_np))

Epoch 1/10
40000/40000 [==============================] - 166s 4ms/step - loss: 0.5169 - accuracy: 0.7414 - val_loss: 0.6098 - val_accuracy: 0.7038
Epoch 2/10
40000/40000 [==============================] - 158s 4ms/step - loss: 0.4911 - accuracy: 0.7541 - val_loss: 0.7817 - val_accuracy: 0.5355
Epoch 3/10
40000/40000 [==============================] - 156s 4ms/step - loss: 0.4872 - accuracy: 0.7557 - val_loss: 0.7792 - val_accuracy: 0.5312
Epoch 4/10
39996/40000 [============================>.] - ETA: 0s - loss: 0.4850 - accuracy: 0.7568